In [1]:
!rm -rf logs/
!rm ngrok*
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-08-17 06:17:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.152.208.69, 54.164.206.44, 52.72.251.164, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.152.208.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  8.42MB/s    in 0.6s    

2018-08-17 06:17:34 (8.42 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = 'logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [4]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://950c1dd5.ngrok.io


In [0]:
from sklearn.datasets import load_iris
import numpy as np

In [0]:
x = load_iris().data
y = load_iris().target

In [0]:
from sklearn.model_selection import train_test_split

In [8]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
y = to_categorical(y)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [0]:
import tensorflow as tf

In [0]:
tf.reset_default_graph()

with tf.name_scope('input'):
  input_features = tf.placeholder(dtype=tf.float32, shape=[None, x.shape[1]],
                                 name='input_features')
  input_labels = tf.placeholder(dtype=tf.float32, shape=[None, y.shape[1]],
                               name='input_labels')

  with tf.name_scope('model'):
    weights = tf.Variable(tf.random_normal(shape=[x.shape[1], y.shape[1]]),
                          name='weights')
    biases = {'hidden_layer':tf.Variable(tf.random_normal(shape=[16]),name = 'weights'),
             'output_layer':tf.Variable(tf.random_normal(shape=[y.shape[1]]), name = ' weights')
             }
    
    hidden_layer = tf.add(tf.matmul(input_features, weights['hidden_layer']),biases['hidden_layer']
    hidden_layer = tf.nn.relu(hidden_layer)
    output_layer = tf.add(tf.matmul(hidden_layer, weights['output_layer']),biases['output_layer'])
    predictions = tf.nn.softmax(output_layer)
    
with tf.name_scope('training_ops'):
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
      logits=linear_model, labels=input_labels))
  train_op = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(loss)
  tf.summary.scalar(name='loss', tensor=loss)

with tf.name_scope('metrics'):
  correct_prediction = tf.cast(tf.equal(tf.argmax(predictions, 1),
                                         tf.argmax(input_labels, 1)), tf.float32)
  accuracy_op = tf.reduce_mean(correct_prediction)
  tf.summary.scalar(name='accuracy', tensor=accuracy_op)

summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(logdir='logs', graph=tf.get_default_graph())

In [0]:
init_op = tf.global_variables_initializer()

In [0]:
def next_batch(batch_size, features, labels):
  indices = np.arange(start=0, stop=features.shape[0])
  np.random.shuffle(indices)
  indices = indices[:batch_size]
  return features[indices], labels[indices]

In [0]:
with tf.Session() as sess:
  sess.run(init_op)
  
  # epochs = 10
  for epoch in range(10):
    for mini_batch in range(int(x_train.shape[0] / 16)):
      batch_x, batch_y = next_batch(batch_size=16, features=x_train, labels=y_train)
      
      _, train_loss, train_accuracy, train_summary = sess.run([train_op, loss,
                                                               accuracy_op,
                                                              summary],
                                              feed_dict={input_features: batch_x,
                                                        input_labels: batch_y})
      writer.add_summary(summary=train_summary, global_step=mini_batch)
    print('Epoch {}, loss : {}, accuracy : {}'.format(epoch, train_loss,
                                                      train_accuracy))  
  
  test_accuracy = sess.run(accuracy_op, feed_dict={input_features: x_test, input_labels: y_test})
  print('Test accuracy : {}'.format(test_accuracy))

Epoch 0, loss : 8.017980575561523, accuracy : 0.25
Epoch 1, loss : 6.987235069274902, accuracy : 0.1875
Epoch 2, loss : 5.915545463562012, accuracy : 0.1875
Epoch 3, loss : 2.958876609802246, accuracy : 0.5
Epoch 4, loss : 3.881527900695801, accuracy : 0.25
Epoch 5, loss : 1.5812339782714844, accuracy : 0.625
Epoch 6, loss : 2.3556437492370605, accuracy : 0.3125
Epoch 7, loss : 1.4876248836517334, accuracy : 0.5
Epoch 8, loss : 1.7820104360580444, accuracy : 0.3125
Epoch 9, loss : 1.0548107624053955, accuracy : 0.5625
Test accuracy : 0.5


In [0]:
with tf.Session() as sess:
  sess.run(init_op)
  prediction_ = sess.run(predictions, feed_dict={input_features: x_test[1].reshape(-1, x_test.shape[1])})
  print(prediction_)
  print('predicted class : {}, true class : {}'.format(sess.run(tf.argmax(prediction_, 1)), sess.run(tf.argmax(y_test[1]))))

[[6.4297562e-04 9.9925894e-01 9.8076918e-05]]
predicted class : [1], true class : 2
